# AutoGuessQuery 测试

本笔记本用于测试 `src/autocoder/agent/auto_guess_query.py` 中的 AutoGuessQuery 类功能。

In [1]:
# 初始化环境
import os
import sys
from autocoder.agent.auto_guess_query import AutoGuessQuery, NextQuery
import byzerllm

In [2]:
# 初始化 LLM 和 AutoGuessQuery
llm = byzerllm.ByzerLLM.from_default_model(model="deepseek_chat")
project_dir = "/Users/allwefantasy/projects/auto-coder"
auto_guesser = AutoGuessQuery(llm, project_dir,skip_diff=True)

2024-12-27 10:01:00.109 | INFO     | byzerllm.utils.connect_ray:connect_cluster:48 - JDK 21 will be used (/Users/allwefantasy/.auto-coder/jdk-21.0.2.jdk/Contents/Home)...
2024-12-27 10:01:00,247	INFO worker.py:1564 -- Connecting to existing Ray cluster at address: 127.0.0.1:6379...
2024-12-27 10:01:00,295	INFO worker.py:1740 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


In [7]:
# 测试 parse_history_tasks 方法
history_tasks = auto_guesser.parse_history_tasks()
print(f"Parsed {len(history_tasks)} history tasks")
for task in history_tasks:
    print(f"Query: {task[0]}")
    print(f"Files: {task[1]}")
    print(f"Diff: {task[2][:100]}..." if task[2] else "No diff")
    print("-" * 40)

Parsed 100 history tasks
Query: 参考 @test_auto_filegroup.ipynb 生成对 @src/autocoder/agent/auto_guess_query.py的 jupyter notebook
Files: ['/Users/allwefantasy/projects/auto-coder/src/autocoder/agent/auto_filegroup.py', '/Users/allwefantasy/projects/auto-coder/src/autocoder/agent/auto_guess_query.py', '.auto-coder/libs/llm_friendly_packages/github.com/allwefantasy/byzer-llm/README.md']
No diff
----------------------------------------
Query: 参考 @src/autocoder/agent/auto_filegroup.py 实现一个 auto_guesss_query.py 该文件是根据历史的需求列表,每个需求对应的Urls以及diff信息,得到下一步要做的需求以及可能的文件列表,使用prompt 函数来生成需求以及urls, prompt 函数返回json str, 并且提供对应的 pydantic model
Files: ['/Users/allwefantasy/projects/auto-coder/src/autocoder/agent/auto_filegroup.py', '.auto-coder/libs/llm_friendly_packages/github.com/allwefantasy/byzer-llm/README.md']
No diff
----------------------------------------
Query: 参考 @src/autocoder/agent/auto_filegroup.py 实现一个 auto_guesss_query.py 该文件是根据历史的需求列表,每个需求对应的Urls以及diff信息,得到下一步要做的需求以及可能的文件列表,使用prompt 函数来生成需求以及

In [9]:
# 测试 predict_next_task 方法
next_task = auto_guesser.predict_next_task()
if next_task:
    print(f"Next Query: {next_task.query}")
    print(f"Predicted Files: {next_task.urls}")
    print(f"Priority: {next_task.priority}")
    print(f"Reason: {next_task.reason}")
    print(f"Dependencies: {next_task.dependency_queries}")
else:
    print("No next task predicted")

Traceback (most recent call last):
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/pydantic/main.py", line 1097, in parse_raw
    obj = parse.load_str_bytes(
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/site-packages/pydantic/deprecated/parse.py", line 49, in load_str_bytes
    return json_loads(b)  # type: ignore
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/miniconda3/envs/byzerllm/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/allwefantasy/projects/auto-

No next task predicted


In [ ]:
# 测试 NextQuery 模型
sample_next_query = NextQuery(
    query="Implement new feature X",
    urls=["src/feature_x.py", "tests/test_feature_x.py"],
    priority=3,
    reason="Required for upcoming release",
    dependency_queries=["Setup basic project structure"]
)
print(sample_next_query.json(indent=2))

## 测试说明

1. `parse_history_tasks` 测试：验证是否能正确解析历史任务
2. `predict_next_task` 测试：验证是否能正确预测下一个任务
3. `NextQuery` 模型测试：验证数据模型是否能正常工作

注意：测试前请确保在 `tests/resources/test_project/actions` 目录下存在有效的 YAML 文件。